In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf, col
from pyspark.ml.feature import StopWordsRemover, Tokenizer
from analysis import WordList
from pyspark.sql.types import IntegerType, FloatType

In [2]:
spark = SparkSession \
        .builder \
        .master('spark://10.10.28.172:7077') \
        .appName('sentiment_wordlist') \
        .enableHiveSupport() \
        .config("spark.pyfiles", "analysis.py") \
        .getOrCreate()

# Preventing errors of having unused settings in hive-site.xml
spark.sparkContext.setLogLevel('OFF')
# path of dependency file(s)
spark.sparkContext.addPyFile("/home/ubuntu/twitter_sentiment/code/analysis/analysis.py")

spark.sql('show databases').show()

22/04/11 09:32:26 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


+------------+
|   namespace|
+------------+
|     default|
|twitter_data|
+------------+



In [5]:
spark.sql('use twitter_data')
spark.sql('show tables').show()

+------------+--------------------+-----------+
|    database|           tableName|isTemporary|
+------------+--------------------+-----------+
|twitter_data|      processed_data|      false|
|twitter_data|            raw_data|      false|
|twitter_data|             results|      false|
|twitter_data|temp_sentiment_re...|      false|
|twitter_data|    textblob_results|      false|
|twitter_data|       vader_results|      false|
|twitter_data|wordlist2477_results|      false|
+------------+--------------------+-----------+



In [6]:
df = spark.sql('select * from processed_data')

In [24]:
fst = df.take(1)
text = fst[0]['text']
text

'ive deleted my tweet about showing kharkiv as part of russia instead of ukraine fortunately turns out to have been a fake not an error at cnn'

In [29]:
wl = WordList(r'/home/ubuntu/twitter_sentiment/code/analysis/wordlists/wl2477.txt')
wl.analyze(text)

udf_wl_analyze = udf(lambda text: wl.analyze(text), IntegerType())

In [30]:
df.withColumn('sentiment', udf_wl_analyze('text')).select('text', 'sentiment').show(1, truncate=False)

+---------------------------------------------------------------------------------------------------------------------------------------------+---------+
|text                                                                                                                                         |sentiment|
+---------------------------------------------------------------------------------------------------------------------------------------------+---------+
|ive deleted my tweet about showing kharkiv as part of russia instead of ukraine fortunately turns out to have been a fake not an error at cnn|-5       |
+---------------------------------------------------------------------------------------------------------------------------------------------+---------+
only showing top 1 row

